In [120]:
!pip install plotly
!pip install lightgbm
!pip install --upgrade pip
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from IPython.display import Image, display
from pandas.tools.plotting import scatter_matrix 
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn import preprocessing
import lightgbm as lgb 
import datetime

Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (18.1)


In [121]:
train_df = pd.read_csv("/Users/sandy/Documents/ENPM808/Project/Github/google-revenue-prediction/train.csv")
#train_df["date"] = pd.to_datetime(train_df["date"], format="%Y%m%d")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [122]:
train_df.head()

,channelGrouping,date,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,Firefox,desktop,False,Windows,9499785259412240342,not available in demo dataset,Europe,Netherlands,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472812272,1.0,1.472812e+09
1,Organic Search,20160902,Chrome,desktop,False,Windows,3696906537737368442,Buenos Aires,Americas,Argentina,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472856874,1.0,1.472857e+09
2,Organic Search,20160902,Chrome,desktop,False,Linux,5688512451361930466,Bengaluru,Asia,India,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472811524,1.0,1.472812e+09
3,Organic Search,20160902,Safari,mobile,True,iOS,9716453263501147451,New Delhi,Asia,India,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472823508,1.0,1.472824e+09
4,Organic Search,20160902,Chrome,desktop,False,Windows,5330453591145100432,Sao Paulo,Americas,Brazil,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472839619,1.0,1.472840e+09


In [128]:
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
#test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float 
# scikit.rf needs numerical data. One hot encoding is not good on rf.
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')))
    
    #lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    #test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


num_cols = ["fullVisitorId", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    #test_df[col] = test_df[col].astype(float)

# Split the train dataset into development and valid based on time 
dev_df = train_df[train_df['date']<=20170531]
val_df = train_df[train_df['date']>20170531]
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)


dev_X = dev_df.loc[:, dev_df.columns != 'totals.transactionRevenue']
val_X = val_df.loc[:, val_df.columns != 'totals.transactionRevenue']

#dev_X = dev_df[cat_cols + num_cols] 
#val_X = val_df[cat_cols + num_cols] 

#test_X = test_df[cat_cols + num_cols] 

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [129]:
#NaNs return error in random forest.
dev_X.fillna(0, inplace=True)
val_X.fillna(0, inplace=True)

dev_X.head()

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,channelGrouping,date,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime
0,4,20160902,93,0,False,5,9.499785e+18,947,4,175,...,3,20,1,3574,5,1605,273,1472812272,1.0,1.472812e+09
1,4,20160902,83,0,False,5,3.696907e+18,335,2,132,...,3,20,1,3574,5,1605,273,1472856874,1.0,1.472857e+09
2,4,20160902,83,0,False,13,5.688512e+18,892,3,137,...,3,20,1,3574,5,1605,273,1472811524,1.0,1.472812e+09
3,4,20160902,128,1,True,8,9.716453e+18,480,3,137,...,3,20,1,3574,5,1605,273,1472823508,1.0,1.472824e+09
4,4,20160902,83,0,False,5,5.330454e+18,724,2,56,...,3,20,1,3574,5,1605,273,1472839619,1.0,1.472840e+09


In [130]:
list(train_df)

['channelGrouping',
 'date',
 'device.browser',
 'device.deviceCategory',
 'device.isMobile',
 'device.operatingSystem',
 'fullVisitorId',
 'geoNetwork.city',
 'geoNetwork.continent',
 'geoNetwork.country',
 'geoNetwork.metro',
 'geoNetwork.networkDomain',
 'geoNetwork.region',
 'geoNetwork.subContinent',
 'totals.bounces',
 'totals.hits',
 'totals.newVisits',
 'totals.pageviews',
 'totals.sessionQualityDim',
 'totals.timeOnSite',
 'totals.totalTransactionRevenue',
 'totals.transactionRevenue',
 'totals.transactions',
 'trafficSource.adContent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaign',
 'trafficSource.isTrueDirect',
 'trafficSource.keyword',
 'trafficSource.medium',
 'trafficSource.referralPath',
 'trafficSource.source',
 'visitId',
 'visitNumber',
 'visitStartTime']

In [131]:
# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    #pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return model, pred_val_y

# Training the model #
#pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)
model, pred_val_lgb = run_lgb(dev_X, dev_y, val_X, val_y)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.0321245
[200]	valid_0's rmse: 0.0286279
[300]	valid_0's rmse: 0.0271579
[400]	valid_0's rmse: 0.0264237
[500]	valid_0's rmse: 0.0259488
[600]	valid_0's rmse: 0.0253996
[700]	valid_0's rmse: 0.0250827
[800]	valid_0's rmse: 0.0248851
[900]	valid_0's rmse: 0.0245123
[1000]	valid_0's rmse: 0.0244208
Did not meet early stopping. Best iteration is:
[978]	valid_0's rmse: 0.0244089


In [132]:
#Calculate RMSE
from sklearn import metrics
pred_val_lgb[pred_val_lgb<0] = 0
val_pred_lgb = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_lgb["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_lgb["PredictedRevenue"] = np.expm1(pred_val_lgb)
#print(np.sqrt(metrics.mean_squared_error(np.pred_val_lgb(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_lgb = val_pred_lgb.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_lgb["transactionRevenue"].values), np.log1p(val_pred_lgb["PredictedRevenue"].values))))

0.024126284564867734


In [133]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees


def run_rf(train_X, train_y, val_X, val_y):
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
    rf.fit(train_X, train_y);
    
    pred_val_y = rf.predict(val_X)
    return rf, pred_val_y
# Train the model on training data
rf, pred_val_rf = run_rf(dev_X, dev_y, val_X, val_y);

In [134]:
from sklearn import metrics
pred_val_rf[pred_val_rf<0] = 0
val_pred_rf = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_rf["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_rf["PredictedRevenue"] = np.expm1(pred_val_rf)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_rf = val_pred_rf.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_rf["transactionRevenue"].values), np.log1p(val_pred_rf["PredictedRevenue"].values))))

0.018811369574266552
